In [1]:
from data import Imdb
from explainers import *
from metrics import *
import numpy as np
import pandas as pd
n_features = 10
imdb = Imdb(n_features=n_features)

Trying to unpickle estimator LinearSVC from version 1.4.1.post1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator _SigmoidCalibration from version 1.4.1.post1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator CalibratedClassifierCV from version 1.4.1.post1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator CountVectorizer from version 1.4.1.post1 when using version 1.4.0. This might lead t

In [3]:
exp = QIIExplainer(imdb)
exp()

Percent: [####################] 100% Count: 600 Approx: 74.97 | Done...


[('even', 0.09453888888888894),
 ('like', 0.0758194444444444),
 ('would', 0.05720277777777776),
 ('movie', 0.01913611111111111),
 ('really', 0.017127777777777787),
 ('good', -0.015174999999999994)]

In [39]:
from qii.qii import QII
from qii.predictor import QIIPredictor
from qii.qoi import QuantityOfInterest


class LRPredictor(QIIPredictor):
    def __init__(self, predictor):
        super(LRPredictor, self).__init__(predictor)

    def predict(self, x):
        # predict the label for instance x
        return self._predictor.predict(x)


lr_predictor = LRPredictor(imdb.model)
quantity_of_interest = QuantityOfInterest()
qii = QII(np.array(imdb.X_vectorized.todense()),
          n_features, quantity_of_interest)

In [58]:
x_0_idx = 42
instance = imdb.X[x_0_idx]
instance = imdb.vectorizer.transform([instance])
instance = np.array(instance.todense())



# Compute Shapley values

banzhaf_vals = qii.compute(x_0=instance, predictor=lr_predictor,
                           show_approx=True, evaluated_features=None,
                           data_exhaustive=False, feature_exhaustive=False,
                           method='banzhaf')
# Replace feature indices with feature names
banzhaf_vals = {imdb.feature_names[i]: banzhaf_vals[i]
                for i in range(len(banzhaf_vals))}
# Sort by feature importance
banzhaf_vals = sorted(banzhaf_vals.items(), key=lambda x: x[1], reverse=True)
print(banzhaf_vals)

Percent: [--------------------] 0% Count: 1 Approx: 0.15 | 

Percent: [####################] 100% Count: 600 Approx: 70.58 | Done...
Banzhaf: 
[('even', 0.16230000000000003), ('would', 0.05942222222222218), ('story', 0.04620277777777775), ('like', 0.021799999999999993), ('good', 0.01841111111111111), ('really', 0.009330555555555555), ('film', -0.01347777777777777), ('movie', -0.018247222222222227), ('time', -0.019927777777777766), ('one', -0.14817222222222232)]


In [ ]:
%matplotlib inline
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

features = range(n_features)
influence_scores = list(banzhaf_vals.values())
fig, ax = plt.subplots()
plt.bar(features, influence_scores)
plt.xticks(features, feature_names, rotation=45, ha="right")
plt.title('Shapley value of data point ' + str(x_0_idx))
plt.show()